>completed this notebook  4 Aug '18 :)

>[Reference1](https://www.youtube.com/watch?v=OAl6eAyP-yo)

>[Reference2](https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction)

## the evaluation metric

- ROC curve, plot of *True positive rate* on Y axis versus *False positive rate*, true positive means how many times the classifier predicted positive when the actual value was positive, the false positive means hown many times the actual result is negative when we predicted the positive.

- Both the true positive rate and false positive rate range from 0 to 1

- to generate the entire ROC curve we have to plot the True positive rates versus false positive rate for the entire range of Threshold from 1 to 0

- ROC curve visualizes the misclassification for every threshold while the singlew misclassification metric evaluates the misclassification for a single threshold

- The thresholds used to generate the ROC are not visible on the ROC curve itself

- A better classifier has the ROC curve on the far upper left corner away from the line (x=y, the random classifier) where as the poor classifier has the ROC curve close to the line (x=y,representing the random classifier) .

- thus for the ease of a single valued evaluation metric we consider the area under the AUC curve ,it follows from the above stated point that the good classifier has area more close to 1 (e.g. 0.8,0.9..or so) while a poor classifier has the area close to 0.5 (e.g. 0.55,0.58,0.6..or so).

- even when the classes are not balanced (as in the case of this competition when the two classes are divided as 92% and 8%) the ROC gives the correct measure of misclassification.

- the ROC curve gives the correct measure even when the range of the probabilities predicted is on different scale (0.9 to 1 or some other scale) other than 0 to 1 as long as the ordering of the classification is preserved. Meaning that the ROC curve is only sensitive to rank ordering

- ROC AU is the probability that the classifier will rank the randomly chosen positive obeservation higher than the randomly chosen negative observation.

- ROC curve can be used to represent more than two classes using the 1 versus all approach.

    - that is if we have 3 classes then we can have 3 ROC curves showing 1 vs (2 and 3), 2 vs (1and 3), 3 vs (1 and 2)
    
- Choosing the threshold based on the ROC curve is a business decision ( whether we want to Maximize true positive or we want to minimise false positive)

    - e.g. If we want to flag a fraudulant credit card transaction then we might want to maximize the False positive so as to avoid missing even a single case of fraudulant transaction.

In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Training data
app_train = pd.read_csv('../input/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
# Testing data features
app_test = pd.read_csv('../input/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

In [ ]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Missing values statistics
missing_values = missing_values_table(app_train)
missing_values.head(20)

> in the earlier notebook we have already found the number of unique labels and the datatype of each columns

>while xtreme gradient boosting machine takes care of the null values the lgbm takes care of the categorical features
>the approach to encode the categorical features is either label encoding where each unique label is assigned a unique integer but then we can also use the one hot encoding where each unique label is assigne a column and then the value 1 and 0 is used to indicate its presence and absence.
>for the label encoding if there are only two categories then it is safe to use it but when there are more than two categories then the model may assign arbitrary weight to the categories which we dont want hence, we prefer the one hot encoding for more than one unique labels

>in case of the exploding number or the number of features after one hot coding we can use the PCA to or any other dimension reduction technique to to reduce the number of columns
>here we will use the feature encoding for only two unique labels and then one hot encoding for more than two unique labels

>we use the scikit learn labelencoder for label encoding and the pandas dummy variable for the one hot label encoding.


In [ ]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

In [ ]:
train_labels = app_train["TARGET"]

In [ ]:
#aligning the two data frames to keep only the colkumns that are there in both the dataframes
app_train,app_test= app_train.align(app_test,join='inner',axis=1)
#adding back the target
app_train["TARGET"] = train_labels
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

> now we have equal number of features in the training dataframe as well as the test dataframe

## Exploratory data analysis

> now here we change the days of birth to the year 

In [ ]:
(app_train['DAYS_BIRTH']/-365).describe()

In [ ]:
app_train['DAYS_EMPLOYED'].describe()

## finding the anomalies in the *DAYS_EMPLOYED* column

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
anom = app_train[app_train['DAYS_EMPLOYED'] == 365243]
non_anom = app_train[app_train['DAYS_EMPLOYED'] != 365243]
print('The non-anomalies default on %0.2f%% of loans' % (100 * non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

> an approach to flag the anomalies

In [ ]:
#boolean flagging the anomaly of the "number of employement days"
app_train['DAYS_EMPLOYED_ANOMALIES'] = app_train['DAYS_EMPLOYED']== 365243
#now we replace the anomalies values with the NAN
app_train['DAYS_EMPLOYED'].replace({365243:np.nan},inplace = True)
app_train['DAYS_EMPLOYED'].plot.hist(title = 'days employement histogram')
plt.xlabel("Days of Employement")


> we have to do the necessary changes to the test data also

In [ ]:
app_test['DAYS_EMPLOYED_ANOMALIES'] = app_test['DAYS_EMPLOYED']==365243
app_test['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

#printing the total number of anomalies 
print("There are %d number of anomalie in the total of %d entries"%(app_test['DAYS_EMPLOYED_ANOMALIES'].sum(),len(app_test)))

> Though the correlation is not the right measure of the dependence between the variable and the target, it does gives the idea about what is effectively more important than the other.

In [ ]:
correlation = app_train.corr()['TARGET']

In [ ]:
correlation = correlation.sort_values()

In [ ]:
#the columns with maximum correlation
correlation.tail(15)

In [ ]:
#the columns with the least correlation are
correlation.head(15)

* .00-.19 “very weak”
*  .20-.39 “weak”
*  .40-.59 “moderate”
*  .60-.79 “strong”
* .80-1.0 “very strong”


## the correlation between the DAYS_Of_Birth

In [ ]:
#here we check the correlation after the values of the columns are changed to absolute values
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

## as the client grows older he tends to pay the loan more timely

In [ ]:
# Set the style of plots
#plt.style.use('fivethirtyeight')

# Plot the distribution of ages in years
plt.hist(app_train['DAYS_BIRTH'] / 365, edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count');

>the histogram does not give us much detail about the distribution of the age of the client other than the obeservation that there are no outliners

>now we observe the KDE plot it is nothing but the histogram that had been smoothed it is usually calculated by finding the gaussian kernel at every point and then smoothing them to get the plot

In [ ]:
plt.figure(figsize=(10,8))
#the loans that were paid on time
sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH']/365,label='target=0')
#the loans that were not paid on time
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target=1')
plt.xlabel('Age(Year)'),plt.ylabel('Density'),plt.title('distribution of Ages')

>we can see that here the plot of target==1 is a bit skewed towards the younger age, though it does not tells us much about the correlation(-0.07) but we know that it does affects the target value in some way so now we further explore the variable

>now we will look at the failure to pay the loan by the age bracket

>to plot this graph we divide the age category into the bins of 5 years each and then we calculate the average payment by each group and then it in turn gives the ratio of the loan that was not paid by each category

In [ ]:
#we create a separate dataframe with the age information and the target
age_data = app_train[['TARGET','DAYS_BIRTH']]
age_data['YEAR_BIRTH'] = age_data['DAYS_BIRTH']/365


In [ ]:
# Bin the age data
#we knew that there the range of age is between (20,70) and thus we have divided the age into bins and then the appropriate bin was assigned to each row
age_data['YEARS_BINNED'] = pd.cut(age_data['YEAR_BIRTH'], bins = np.linspace(20, 70, num = 11))
age_data.head(10)

In [ ]:
age_data.head()

In [ ]:
age_group = age_data.groupby("YEARS_BINNED").mean()
age_group

>now here we plot the age bracketwise distribution of the loan repayment and if any delay in loan repayments

In [ ]:
plt.figure(figsize = (8, 8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_group.index.astype(str), 100 * age_group['TARGET'])

# Plot labeling
plt.xticks(rotation = 75); plt.xlabel('Age Group (years)'); plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');

>here we can very well see that the younger clients tend to have more delay as compared to the  older clients, the younger clients should be given the loan with more precaution

## now we have some external data source which in some sense is the measure of the credit

>we explore the correlation between different columns of the external data sources

In [ ]:
external_data = app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]

In [ ]:
external_DataCorr = external_data.corr()

In [ ]:
external_DataCorr

>it can be seen that as the value of the value of the EXT_SOURCE 1,2 and 3 will increase the clients are more likely to repay the loan

In [ ]:
plt.figure(figsize = (8, 6))

# Heatmap of correlations
sns.heatmap(external_DataCorr, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

In [ ]:
plt.figure(figsize = (10, 12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']):
    
    # create a new subplot for each source
    plt.subplot(3, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, source], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, source], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' % source); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

In [ ]:
# Copy the data for plotting
plot_data = external_data.drop(columns = ['DAYS_BIRTH']).copy()

# Add in the age of the client in years
plot_data['YEARS_BIRTH'] = age_data['YEAR_BIRTH']

# Drop na values and limit to first 100000 rows
plot_data = plot_data.dropna().loc[:100000, :]

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, size = 3, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha = 0.2)

# Diagonal is a histogram
grid.map_diag(sns.kdeplot)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot', size = 32, y = 1.05);

In [ ]:
## Make a new dataframe for polynomial features
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]


# imputer for handling missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

poly_target = poly_features['TARGET']

poly_features = poly_features.drop(columns = ['TARGET'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
#poly_features = imputer.fit_transform(poly_features)
#the fit method calculates the internal parameters to be used for the transformation and saves as an internal object
#the transformation transforms the values using the value calculated by the fit function
#the fit_transform does the two things to a dataset at the same time
#poly_features_test = imputer.transform(poly_features_test)


In [ ]:
# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

In [ ]:
poly_transformer.get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

In [ ]:
# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Add in the target
poly_features['TARGET'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['TARGET'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# Merge polynomial features into training dataframe
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

## so now we have some features, we will try and see if these features will improve the model

In [ ]:
#using some of the domain knowledge
app_train_domain =  app_train.copy()
app_test_domain = app_test.copy()


In [ ]:
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

In [ ]:
plt.figure(figsize = (12, 20))
# iterate through the new features
for i, feature in enumerate(['CREDIT_INCOME_PERCENT', 'ANNUITY_INCOME_PERCENT', 'CREDIT_TERM', 'DAYS_EMPLOYED_PERCENT']):
    
    # create a new subplot for each source
    plt.subplot(4, 1, i + 1)
    # plot repaid loans
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET'] == 1, feature], label = 'target == 1')
    
    # Label the plots
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

## The only way to say if some faeture will work is to try them 

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()

In [ ]:
# Feature names
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

In [ ]:
# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

In [ ]:
# Repeat with the scaler
#the fit method calculates the minimum and maximum needed to compute the transformed value between 0 and 1
#the transform function transforms the value using the calculated minimum and the maximum
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

In [ ]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [ ]:
# Submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

In [ ]:
# Save the submission to a csv file
submit.to_csv('log_reg_baseline.csv', index = False)

## Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [ ]:
# Train on the training data
random_forest.fit(train, train_labels)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict_proba(test)[:, 1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline.csv', index = False)

## Using the engineered feature

In [ ]:
poly_features_names = list(app_train_poly.columns)

# Impute the polynomial features
imputer = Imputer(strategy = 'median')

poly_features = imputer.fit_transform(app_train_poly)
poly_features_test = imputer.transform(app_test_poly)

# Scale the polynomial features
scaler = MinMaxScaler(feature_range = (0, 1))

poly_features = scaler.fit_transform(poly_features)
poly_features_test = scaler.transform(poly_features_test)

random_forest_poly = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [ ]:
app_train_domain = app_train_domain.drop(columns = 'TARGET')

domain_features_names = list(app_train_domain.columns)

# Impute the domainnomial features
imputer = Imputer(strategy = 'median')

domain_features = imputer.fit_transform(app_train_domain)
domain_features_test = imputer.transform(app_test_domain)

# Scale the domainnomial features
scaler = MinMaxScaler(feature_range = (0, 1))

domain_features = scaler.fit_transform(domain_features)
domain_features_test = scaler.transform(domain_features_test)

random_forest_domain = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

# Train on the training data
random_forest_domain.fit(domain_features, train_labels)

# Extract feature importances
feature_importance_values_domain = random_forest_domain.feature_importances_
feature_importances_domain = pd.DataFrame({'feature': domain_features_names, 'importance': feature_importance_values_domain})

# Make predictions on the test data
predictions = random_forest_domain.predict_proba(domain_features_test)[:, 1]

In [ ]:
# Make a submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = predictions

# Save the submission dataframe
submit.to_csv('random_forest_baseline_domain.csv', index = False)

In [ ]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
# Show the feature importances for the default features
feature_importances_sorted = plot_feature_importances(feature_importances)

In [ ]:
feature_importances_domain_sorted = plot_feature_importances(feature_importances_domain)

## This notebook concludes thanks @willkoehrsen